In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import numpy as np
import matplotlib.pyplot as plt
import math
import random
import h5py
import copy
from scipy.optimize import curve_fit


# In[2]:


class Sig:
    def __init__(self):
        self.number_groups = 0
        self.sig_f = []
        self.capture = []
        self.scattering = []
        self.total = []
        self.number_of_production_neutrons = []
        self.c_value = []
        self.virtual = []
        
    def __init__(self, number_groups, sig_f, sig_c, sig_s, number_of_production_neutrons, sig_t):
        self.number_groups = number_groups
        self.sig_f = sig_f
        self.sig_c = sig_c
        self.sig_s = sig_s      
        self.number_of_production_neutrons = number_of_production_neutrons
        self.sig_t = sig_t
        self.delta_xs = max(sig_t)/2.
        max_total_cs = max(self.sig_t)
        self.virtual = [self.delta_xs]
        
               
    def get_virtual_cs(self):
        max_total_cs = max(self.sig_t)
        self.virtual = [cross_section - max_total_cs + self.delta_xs for cross_section in self.sig_t]
        
    def return_virtual(self):
        return self.virtual
        
    def get_fission_probability(self, energy_group_idx):
        return self.sig_f[energy_group_idx] / self.sig_t[energy_group_idx]
    
    def get_capture_probability(self, energy_group_idx):
        return self.sig_c[energy_group_idx] / self.sig_t[energy_group_idx]
        
    def get_scatter_probability(self, energy_group_idx):
        return self.sig_s[energy_group_idx] / self.sig_t[energy_group_idx]
        


# In[3]:


energy_groups_2 = [0, 2 * 1E10]

class Positon:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
class Direction:
    def __init__(self, ets, phi):
        self.tetta_x = math.sin(ets) * math.cos(phi)
        self.tetta_y = math.sin(ets) * math.sin(phi)
        self.tetta_z = math.cos(ets)  


# In[4]:


class Particle:
    def __init__(self):
        self.coordinates = Positon(0, 0, 0)
        self.direction = Direction(0, 0)
        self.energy = 0
        self.weight = 1
        self.energy_groups = []
        self.terminated = False
        self.path = 0.
        self.last_cell = 0
        
    def set_coordinates(self, x, y, z):
        self.coordinates = Positon(x, y, z)
        
    def set_direction(self, ets, phi):
        self.direction = Direction(ets, phi)
    
    def set_direction_angle(self, angle):
        self.direction = angle
        
    def set_energy_groups(self, energy_groups):
        self.energy_groups = energy_groups
        
    def get_energy_group(self):  
        res = next(x for x, val in enumerate(self.energy_groups)
                                  if val > self.energy)        
        return res - 1
    
    def set_terminated(self):
        self.terminated = True
        
    def is_terminated(self):
        return self.terminated
        
    def get_weight(self):
        return self.weight
    
    def set_weight(self, weight):
        self.weight = weight
    
    def set_particle_deleted(self):
        self.weight = self.weight * 0
        
    def set_particle_fission(self, additional_weight):
        self.weight = self.weight * additional_weight
        
    def set_multiplicity(self, additional_weight):
        self.weight = self.weight * additional_weight
              
    def is_particle_deleted(self):
        return self.weight < 0.0001
    
    def add_path(self, path):
        self.path += path
    
    def get_path(self):
        return self.path
    
    def set_path(self, path):
        self.path = path
        
    def print_direction(self):
        print(str(self.direction.tetta_x) + " " + str(self.direction.tetta_y) + " " + str(self.direction.tetta_z))
        
    def print_coordinates(self):
        print(str(self.coordinates.x) + "  " + str(self.coordinates.y) + " " + str(self.coordinates.z))  
        


# In[5]:


class Plane:
    def __init__(self, A, B, C, D):
        self.A = A
        self.B = B
        self.C = C
        self.D = D
        
    def distance(self, particle):
        vp = (self.A * particle.direction.tetta_x + self.B * particle.direction.tetta_y + 
              self.C * particle.direction.tetta_z)
        
        
        if (abs(vp) < 1e-9):
            return -1
        
        distance = (self.D - self.A * particle.coordinates.x - self.B * particle.coordinates.y 
                    - self.C * particle.coordinates.z) / vp
        
        return distance 
    
    def get_normal(self):
        sq = math.sqrt(self.A * self.A + self.B * self.B + self.C * self.C)
        a_n = self.A / sq
        b_n = self.B / sq
        c_n = self.C / sq
        return [a_n, b_n, c_n]
    

    def get_sign(self, particle):
      
        sign = (self.A * particle.coordinates.x + self.B * particle.coordinates.y +
        self.C * particle.coordinates.z - self.D)
        
        if sign == 0:
            return sign    
            
        if sign < 0:
            return -1
        else:
            return 1
        
    def get_xml(self):
        
        obj_xml = " Plane with A "+ str(self.A) + " B "+ str(self.B) + " C "+ str(self.C) + " D "+ str(self.D)
        
        return obj_xml
    


# In[6]:


class Universe:
    def __init__(self, cells, xs):
        self.cells = cells
        self.xs = xs
        self.delta_xs = 0.
        
    def calculate_delta_xs(self):
        
        delta_xs = 0.
        for xs in self.xs:
            delta_xs = max(delta_xs, max(xs.sig_t))
        self.delta_xs = delta_xs           
            
    def get_delta_xs(self):
        
        return self.delta_xs
    
    def get_xs_by_coordinates(self, particle):
        
        for i in range(0, len(cells)):
            cell = cells[i]
            if cell.is_inside(particle):
                particle.last_cell = i
                return self.xs[i]
        return []
    
    def is_inside(self, particle):
        
        for cell in self.cells:
            if cell.is_inside(particle):
                return True
        return False
        
    
    def is_boudary_reflective(self, particle):
        
        minumum = np.inf
        idx_of_min = 0.
        for i in range(0, len(self.cells)):
            distance, surface_idx = self.cells[i].get_minimum_distance(particle)
            if abs(distance) < minumum:
                minumum = distance
                idx_of_min = i
        if self.cells[idx_of_min].is_boudary_reflective(particle):
            return True
        else:
            return False
        
    def reflect_particle(self,  particle):
        self.cells[particle.last_cell].reflect_particle(particle)       


# In[7]:


def set_random_direction(particle):
    r = 1.0
    r1 = 1.0
    while (r**2 + r1**2 > 1.0):
        r = 2. * random.uniform(0, 1) - 1.
        r1 = 2. * random.uniform(0, 1) - 1.
        
    direction_x = 2.0 * r**2 + 2. * r1**2 - 1.0  
    particle.direction.tetta_x = direction_x
    particle.direction.tetta_y = r * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))
    particle.direction.tetta_z = r1 * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))


# In[8]:


import random

def make_initial_sources(number_of_paricles, box_size, zero_point, energy=10.0e6):
    
    step_x = box_size[0]/number_of_paricles
    step_y = box_size[1]/number_of_paricles
    step_z = box_size[2]/number_of_paricles
    
    x_coord = zero_point[0]
    y_coord = zero_point[1]
    z_coord = zero_point[2]
    
    sources = []
    for i in range(0, number_of_paricles):
        for j in range(0, number_of_paricles):
            for k in range(0, number_of_paricles):
                current_particle = Particle()
                current_particle.set_coordinates(x_coord + k * step_x, y_coord + j * step_y, z_coord+ i * step_z)
                set_random_direction(current_particle)
                current_particle.energy = energy
                current_particle.set_energy_groups(energy_groups_2)
                sources.append(current_particle)

    return sources


# In[9]:


def get_free_path(particle, delta_xs):
    
    energy_group_idx = particle.get_energy_group()   
    sig_t = delta_xs
    free_path = -math.log(random.uniform(0, 1)) / sig_t
    
    return free_path  


# In[10]:


class Cell:
    def __init__(self):
        self.surfaces = []
        self.boundaries_type = []
        self.signs = []
        self.size = 0
        
    def set_boundaries_type(self, boundaries_type):
        self.boundaries_type = boundaries_type        
        
    def set_box_sizes(self, xm_size, ym_size, zm_size, x_size, y_size, z_size):
        self.xm_size = xm_size
        self.ym_size = ym_size
        self.zm_size = zm_size
        
        self.x_size = x_size
        self.y_size = y_size
        self.z_size = z_size
        
    def set_zero_point(self, x_0, y_0, z_0):
        self.x_0 = x_0
        self.y_0 = y_0
        self.z_0 = z_0
        
    def __init__(self, surfaces, signs):
        self.surfaces = surfaces
        self.signs = signs
        self.size = len(self.signs)
           
    def is_inside(self, particle):
        
        is_inside = True
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                is_inside = False
                
        return is_inside
    
    def is_boudary_reflective(self, particle):
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                if self.boundaries_type[i] == 'reflective':
                    return True
                else:
                    return False
        
    
    def get_minimum_distance(self, particle):
        
        minimum_distance = math.inf
        surface_idx = 0
        for i in range(0, len(self.surfaces)):
            current_distance = abs(self.surfaces[i].distance(particle))
            if current_distance > 0:
                minimum_distance = min(minimum_distance, current_distance)
                surface_idx = i
        return minimum_distance, surface_idx
    
    def get_xml(self):
        
        xml_obj = []
        for i in range(0, self.size):
            xml_obj.append(surfaces[i].get_xml())
            
        return xml_obj
    
    def reflect_particle(self, particle):
        
        if particle.coordinates.x > (self.x_size + self.x_0):
            delta = particle.coordinates.x % self.x_size
            particle.coordinates.x = self.xm_size + math.fabs(delta)
        
        if particle.coordinates.y > (self.y_size + self.y_0):
            
            delta = particle.coordinates.y % self.y_size
            particle.coordinates.y = self.ym_size + math.fabs(delta)
            
            
        if particle.coordinates.z > (self.z_size + self.z_0):
            
            delta = particle.coordinates.z % self.z_size
            particle.coordinates.z = self.zm_size + math.fabs(delta)
        
        
        if particle.coordinates.y < (self.ym_size + self.y_0):
            delta = particle.coordinates.y % self.ym_size
            particle.coordinates.y = self.y_size - math.fabs(delta)
            
            
        if particle.coordinates.x < (self.xm_size + self.x_0):
            delta = particle.coordinates.x % self.xm_size
            particle.coordinates.x = self.x_size - math.fabs(delta)
            
        if particle.coordinates.z < (self.zm_size + self.z_0):
            delta = particle.coordinates.z % self.zm_size
            particle.coordinates.z = self.z_size - math.fabs(delta)
                    


# In[11]:


def move_particle(particle, t):
    new_x = particle.direction.tetta_x * t + particle.coordinates.x
    new_y = particle.direction.tetta_y * t + particle.coordinates.y
    new_z = particle.direction.tetta_z * t + particle.coordinates.z
    particle.set_coordinates(new_x, new_y, new_z)


# In[13]:


def is_collision_virual(particle, c_s, delta_xs):
    
    energy_group_idx = particle.get_energy_group()
    random_number = random.uniform(0, 1)
    
    virtual_cs = delta_xs
    
    total_cs = c_s.sig_t[energy_group_idx]
    
    if total_cs/virtual_cs < random_number:
        return True
    else:
        return False


# In[14]:


def process_virtual_collision(particle, free_path):
    particle.add_path(free_path)


# In[15]:


def process_real_collision(particle, free_path, c_s):
    
    energy_group_idx = particle.get_energy_group()
    weight_before_collision = particle.get_weight()
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]   
    capture_probability = c_s.get_capture_probability(energy_group_idx)
    scatter_probability = c_s.get_scatter_probability(energy_group_idx)  
    fission_probability = c_s.get_fission_probability(energy_group_idx)  
    
    
    type_collision = np.random.choice(['capture', 'scatter', 'fission'], p=[capture_probability, scatter_probability,
                                                                            fission_probability])
    if type_collision == 'capture':
        particle.set_terminated()
        particle.set_weight(0.)
        
    if type_collision == 'scatter':      
        set_random_direction(particle)
        
    if type_collision == 'fission':    
        particle.set_particle_fission(number_of_production_neutrons)
        particle.set_terminated()
        
    return weight_before_collision


# In[16]:


def delete_absorpbed_paricles(particles):
    
    existing_particles = []
    for i in range(0, len(particles)):
        particle = particles[i]
        if particle.get_weight() > 0.000001:
            existing_particles.append(particle)
                     
    return existing_particles


# In[17]:


def process_one_particle_history(particle, universe, estimators):
    
    sum_collisions = 0.
    universe.calculate_delta_xs()
    delta_xs = universe.get_delta_xs()
    
    while not particle.is_terminated():
        current_xs = universe.get_xs_by_coordinates(particle)
        free_path = get_free_path(particle, delta_xs)        
        move_particle(particle, free_path)
        current_xs = universe.get_xs_by_coordinates(particle)
        
        if not universe.is_inside(particle):
            if universe.is_boudary_reflective(particle):
                universe.reflect_particle(particle)
                current_xs = universe.get_xs_by_coordinates(particle)

            else:
                particle.set_terminated()
                particle.set_weight(0.)
                return particle, sum_collisions
                
        for k in estimators:
            k.add_collision(particle)
        
        if is_collision_virual(particle, current_xs, delta_xs):
            process_virtual_collision(particle, free_path)
        else:
            process_real_collision(particle, free_path, current_xs)
        
            
    return particle, sum_collisions


# In[18]:


def get_weights(particles):
    
    weights = []
    for i in range(0, len(particles)):
        weights.append(particles[i].weight)
        
    return weights


# In[19]:


def make_sources(particles):
    for i in range(0, len(particles)):
        particles[i].terminated = False
        set_random_direction(particles[i])       


# In[20]:


def splitting_secound_version(particles):
    
    initial_size = len(particles)
    
    for i in range(0, initial_size):
        particle = particles[i]
        current_weight = particle.get_weight()
        if current_weight > 1:
            n_value = math.floor(current_weight)
            random_value = random.uniform(0, 1)
            if current_weight - n_value >= random_value:
                n_value += 1
            if n_value > 1:
                particle.set_weight(current_weight / n_value)
                for j in range(0, n_value - 1):
                    particles.append(copy.deepcopy(particle))


# In[21]:


def normalise_weights(particles, batch_size):

    sum_weights = 0.
    
    for i in range(0, len(particles)):
        sum_weights += particles[i].get_weight()

    for i in range(0, len(particles)):
        particles[i].weight = (particles[i].weight * batch_size) / sum_weights   
        
    return particles


class Flux_estimator:
    def __init__(self, universe):
        self.boundaries = universe
        self.collision_sum = [0]
        
    def add_collision(self, particle):
        if self.boundaries.is_inside(particle):
            self.collision_sum[-1]  += particle.get_weight()
            



def russian_roulette(weights_previous, particles_current):
    
    for i in range(0, len(particles_current)):
        particle = particles_current[i]
        if particle.weight > 0 and particle.weight < 0.5 and particle.weight < weights_previous[i]:
            probability_terminate = 1. - particle.weight / weights_previous[i]
            random_number = random.uniform(0, 1)
            if probability_terminate >= random_number:
                particle.set_weight(0.)
            else:
                particle.set_weight(weights_previous[i])


# In[23]:


def terminate_outside_particles(batch_particles, universe):
    
    for j in range(0, len(batch_particles)):
        particle = batch_particles[j]
        if not universe.is_inside(particle):
            print("ERROR! outside particle")


# In[24]:


def reset_estimators(estimators, weight_previos, volume, sum_collisions, c_s):
    
    sig_t = c_s.sig_t[0]

    for estimator in estimators:
        estimator.collision_sum[-1] = estimator.collision_sum[-1]
        estimator.collision_sum.append(0.)  


# In[25]:


def get_std(values):
    
    current_std = np.std(values)/np.sqrt((len(values) - 1.))
  
    return current_std   


# In[26]:


def calculate_k_effective(idx, weights, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, std_k_effective):
    
    keff_cycle = sum(weights) / initial_size
    if idx > number_inactive:
        k_effective.append(keff_cycle)
        
    if idx > number_inactive + 1:
    
        k_effective_exp.append(sum(k_effective) / len(k_effective))
        
        std_k_effective_current = get_std(k_effective)
            
        std_k_effective.append(std_k_effective_current)

        print(" keff_cycle , k_effective_exp, std_k_effective " + str(keff_cycle) + "   " + str(k_effective_exp[-1]) +
                                    "  "+ str(std_k_effective[-1]))


# In[27]:


import statistics
import time


def simulation_black_boundaries(universe, number_interations, number_inactive, number_of_particles, c_s, estimators,
                                volume=1):
    
    random.seed(time.time())
    k_effective = []
    k_effective_std = []
    k_effective_exp = []
    flux = []
    flux_exp = []
    num1 = random.randint(0,9)
    
    print(" num1  " + str(num1))
    
    box_size = [1., 1., 1.]
    zero_point = [0.1, 0.1, 0.1]
    initial_sources = make_initial_sources(number_of_particles, box_size, zero_point, energy=10.0e6)
    initial_size = len(initial_sources)
    
    weights_previous = [1.] * len(initial_sources)
    
    for i in range(0, number_interations):
        print("i == " + str(i))
        
        make_sources(initial_sources)
        batch_size = len(initial_sources)
        
        batch_particles = []

        for j in range(0, batch_size):
   
            particle = initial_sources[j]
            terminate_particle, sum_collisions = process_one_particle_history(particle, universe, estimators)
            batch_particles.append(terminate_particle)
        
        russian_roulette(weights_previous, batch_particles)
        
        batch_particles = delete_absorpbed_paricles(batch_particles)
        
        splitting_secound_version(batch_particles)
        
        weights_cycle = get_weights(batch_particles)
        
        calculate_k_effective(i, weights_cycle, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, k_effective_std)
        
        
        reset_estimators(estimators, weights_previous, volume, sum_collisions, c_s) 
      

        batch_particles = normalise_weights(batch_particles, initial_size)  

        initial_sources = batch_particles
        weights_previous = get_weights(batch_particles)

            
    return k_effective, k_effective_exp, k_effective_std, estimators


# In[28]:


cs_fission_u235 = [0.06922744]

cs_capture_u235 = [0.01013756]

cs_scattering_u235 = [0.328042]

cs_total_u235 = [0.407407]

cs_production_neutrons_u235 = [2.50]


u235 = Sig(1, cs_fission_u235, cs_capture_u235, cs_scattering_u235, 
                       cs_production_neutrons_u235,  cs_total_u235)


cs_fission_fe = [0.]

cs_capture_fe = [0.00046512]

cs_scattering_fe = [0.23209488]

cs_total_fe = [0.23256]

cs_production_neutrons_fe = [0.]

fe = Sig(1, cs_fission_fe, cs_capture_fe, cs_scattering_fe, 
                       cs_production_neutrons_fe,  cs_total_fe)


cs_fission_na = [0.]

cs_capture_na = [0.0]

cs_scattering_na = [0.086368032]

cs_total_na = [0.086368032]

cs_production_neutrons_na = [0.]

na = Sig(1, cs_fission_na, cs_capture_na, cs_scattering_na, 
                       cs_production_neutrons_na,  cs_total_na)



energy_groups_2 = [0, 2 * 1E10]


# In[29]:


x_5 = Plane(1, 0, 0, 1000)
x_m5 = Plane(1, 0, 0, -1000)

y_5 = Plane(0, 1, 0, 0.317337461)

y_m5 = Plane(0, 1, 0, 0.)


z_5 = Plane(0, 0, 1, 1000)
z_m5 = Plane(0, 0, 1, -1000)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

signs = [-1, +1, -1, +1, -1, +1]
slab_fe = Cell(surfaces, signs)

boundaries_type = ["reflective", "reflective","black", "black", "reflective", "reflective"]


slab_fe.set_boundaries_type(boundaries_type)
slab_fe.set_box_sizes(-1000, 10900., -1000, 1000, 20000., 1000 )
slab_fe.set_zero_point(0., 0., 0.)


# In[30]:


x_5 = Plane(1, 0, 0, 1000)
x_m5 = Plane(1, 0, 0, -1000)

y_5 = Plane(0, 1, 0, 5.437057544)

y_m5 = Plane(0, 1, 0, 0.317337461)


z_5 = Plane(0, 0, 1, 1000)
z_m5 = Plane(0, 0, 1, -1000)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

signs = [-1, +1, -1, +1, -1, +1]
slab_u235 = Cell(surfaces, signs)

boundaries_type = ["reflective", "reflective","black", "black", "reflective", "reflective"]


slab_u235.set_boundaries_type(boundaries_type)
slab_u235.set_box_sizes(-1000, -10000., -1000, 1000, 10000., 1000 )
slab_u235.set_zero_point(0., 0., 0.)


# In[31]:


x_5 = Plane(1, 0, 0, 1000)
x_m5 = Plane(1, 0, 0, -1000)

y_5 = Plane(0, 1, 0, 5.754395005)

y_m5 = Plane(0, 1, 0, 5.437057544)


z_5 = Plane(0, 0, 1, 1000)
z_m5 = Plane(0, 0, 1, -1000)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

signs = [-1, +1, -1, +1, -1, +1]
slab_fe2 = Cell(surfaces, signs)

boundaries_type = ["reflective", "reflective","black", "black", "reflective", "reflective"]


slab_fe2.set_boundaries_type(boundaries_type)
slab_fe2.set_box_sizes(-1000, -10000., -1000, 1000, 10000., 1000 )
slab_fe2.set_zero_point(0., 0., 0.)


# In[32]:


x_5 = Plane(1, 0, 0, 1000)
x_m5 = Plane(1, 0, 0, -1000)

y_5 = Plane(0, 1, 0, 7.757166007)

y_m5 = Plane(0, 1, 0, 5.754395005)


z_5 = Plane(0, 0, 1, 1000)
z_m5 = Plane(0, 0, 1, -1000)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

signs = [-1, +1, -1, +1, -1, +1]
slab_na = Cell(surfaces, signs)

boundaries_type = ["reflective", "reflective","black", "black", "reflective", "reflective"]


slab_na.set_boundaries_type(boundaries_type)
slab_na.set_box_sizes(-1000, -10000.,  -1000, 1000, 10000., 1000 )
slab_na.set_zero_point(0., 0., 0.)


# In[33]:


cells = [slab_fe, slab_u235, slab_fe2, slab_na]


# In[34]:


estimators = []


# In[35]:


materials = [fe, u235, fe, na]


# In[36]:


U235_Reactor = Universe(cells, materials)


# In[37]:



In [65]:
estimators = []

# zero

x_5 = Plane(1, 0, 0, 10000)
x_m5 = Plane(1, 0, 0, -10000)

delta = 0.317337461 * 0.1


y_5 = Plane(0, 1, 0,  delta)

y_m5 = Plane(0, 1, 0, 0)


z_5 = Plane(0, 0, 1, 10000)
z_m5 = Plane(0, 0, 1, -10000)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

signs = [-1, +1, -1, +1, -1, +1]
current_slub = Cell(surfaces, signs)

estimators.append(Flux_estimator(current_slub))

In [22]:
# Fe-U

x_5 = Plane(1, 0, 0, 10000)
x_m5 = Plane(1, 0, 0, -10000)

delta = 0.317337461 * 0.05


y_5 = Plane(0, 1, 0,  0.317337461 + delta)

y_m5 = Plane(0, 1, 0, 0.317337461 - delta)


z_5 = Plane(0, 0, 1, 10000)
z_m5 = Plane(0, 0, 1, -10000)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

signs = [-1, +1, -1, +1, -1, +1]
current_slub = Cell(surfaces, signs)

estimators.append(Flux_estimator(current_slub))

In [23]:
# U-Fe

x_5 = Plane(1, 0, 0, 10000)
x_m5 = Plane(1, 0, 0, -10000)

delta = (0.317337461) * 0.01


y_5 = Plane(0, 1, 0,  5.437057544 + delta)

y_m5 = Plane(0, 1, 0, 5.437057544 - delta)


z_5 = Plane(0, 0, 1, 10000)
z_m5 = Plane(0, 0, 1, -10000)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

signs = [-1, +1, -1, +1, -1, +1]
current_slub = Cell(surfaces, signs)

estimators.append(Flux_estimator(current_slub))

In [24]:
# Fe-Na

x_5 = Plane(1, 0, 0, 10000)
x_m5 = Plane(1, 0, 0, -10000)

delta = (5.754395005 - 5.437057544)  * 0.02


y_5 = Plane(0, 1, 0,  5.754395005 + delta)

y_m5 = Plane(0, 1, 0, 5.754395005 - delta)


z_5 = Plane(0, 0, 1, 10000)
z_m5 = Plane(0, 0, 1, -10000)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

signs = [-1, +1, -1, +1, -1, +1]
current_slub = Cell(surfaces, signs)

estimators.append(Flux_estimator(current_slub))

In [25]:
# Na

x_5 = Plane(1, 0, 0, 10000)
x_m5 = Plane(1, 0, 0, -10000)

delta = (7.757166007 -5.754395005) * 0.05


y_5 = Plane(0, 1, 0,  7.757166007)

y_m5 = Plane(0, 1, 0, 7.757166007 - delta)


z_5 = Plane(0, 0, 1, 10000)
z_m5 = Plane(0, 0, 1, -10000)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

signs = [-1, +1, -1, +1, -1, +1]
current_slub = Cell(surfaces, signs)

estimators.append(Flux_estimator(current_slub))

In [26]:

test_number_of_particles = 20
test_number_interations = 100
test_number_inactive = 50


# In[38]:


k_effective, k_effective_exp, k_effective_std, estimators_result = simulation_black_boundaries(U235_Reactor,  
                           test_number_interations,  test_number_inactive, 
                          test_number_of_particles, u235, estimators)


# In[40]:


def save_simulation_results_to_file(k_effective, file_name):
    
    f = open(file_name, "w")
    
    for i in range(0, len(k_effective)):
        f.write(" cycle k effective "+ str(k_effective[i]) + '\n')
            
    
    f.close()


# In[42]:


file_name = "output.txt"
save_simulation_results_to_file(k_effective, file_name)


 num1  1
i == 0
i == 1
i == 2
i == 3
i == 4
i == 5
i == 6
i == 7
i == 8
i == 9
i == 10
i == 11
i == 12
i == 13
i == 14
i == 15
i == 16
i == 17
i == 18
i == 19
i == 20
i == 21
i == 22
i == 23
i == 24
i == 25
i == 26
i == 27
i == 28
i == 29
i == 30
i == 31
i == 32
i == 33
i == 34
i == 35
i == 36
i == 37
i == 38
i == 39
i == 40
i == 41
i == 42
i == 43
i == 44
i == 45
i == 46
i == 47
i == 48
i == 49
i == 50
i == 51
i == 52
 keff_cycle , k_effective_exp, std_k_effective 1.0039808436888948   1.0112675349984759  0.007286691309580928
i == 53
 keff_cycle , k_effective_exp, std_k_effective 1.0041721803824337   1.0089024167931286  0.0048262208340452955
i == 54
 keff_cycle , k_effective_exp, std_k_effective 0.9931321190114777   1.004959842347716  0.0052144124335448926
i == 55
 keff_cycle , k_effective_exp, std_k_effective 0.9815285398473336   1.0002735818476396  0.006186686972107217
i == 56
 keff_cycle , k_effective_exp, std_k_effective 0.9867533528725545   0.998020210351792  0.005531221705813158


In [27]:
difference = (k_effective_exp[-1] - 1.) * 100000
k_effective_std = get_std(k_effective)
standart_deviation = k_effective_std * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(standart_deviation))

 difference from beachmark [pcm]  -179.19213668191603  with standart deviation [pcm]+- 185.1483807206475


In [28]:
zero_estimator = estimators_result[0]

average_value = sum(zero_estimator.collision_sum)/len(zero_estimator.collision_sum)
zero_point = average_value / (0.317337461 * 0.05 *2 )

12026.519845113924


In [16]:
print(average_value)

0.4695095583413393


In [18]:
Fe_U_estimator = estimators_result[1]

average_value = sum(Fe_U_estimator.collision_sum)/len(Fe_U_estimator.collision_sum)
average_value = average_value / ( zero_point * 0.317337461 * 0.05 * 0.086368032)

average_array = np.array(Fe_U_estimator.collision_sum)
average_array = average_array / (zero_point * 0.317337461 * 0.05 *  0.086368032)
current_std = get_std(average_array)

In [19]:
difference = (average_value - 1.229538) * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(current_std * 100000))

 difference from beachmark [pcm]  -76002.84416586607  with standart deviation [pcm]+- 535.1293435120306


In [20]:
U_Fe_estimator = estimators_result[2]

start = 10
end = len(U_Fe_estimator.collision_sum) - 1

average_value = sum(U_Fe_estimator.collision_sum[start: end])/len(U_Fe_estimator.collision_sum[start: end])
average_value = average_value / (zero_point * 0.317337461 * 0.01)


average_array = np.array(U_Fe_estimator.collision_sum[start: end])
average_array = average_array / (zero_point *  0.317337461 * 0.01)
current_std = get_std(average_array)

In [21]:
difference = (average_value - 1.49712) * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(current_std * 100000))

 difference from beachmark [pcm]  -112153.7049215821  with standart deviation [pcm]+- 326.0365418288852


In [87]:
U_Fe_estimator = estimators_result[3]
start = 10
end = len(U_Fe_estimator.collision_sum) - 1
average_value = sum(U_Fe_estimator.collision_sum[start: end])/len(U_Fe_estimator.collision_sum[start: end])
average_value = average_value / (zero_point * (5.754395005 - 5.437057544)  * 0.04)


average_array = np.array(U_Fe_estimator.collision_sum[start: end])
average_array = average_array / (zero_point * (5.754395005 - 5.437057544)  * 0.04)
current_std = get_std(average_array)

In [89]:
print(U_Fe_estimator.collision_sum[start: end])

[35.046102393494245, 23.965441256148623, 21.550041869601383, 24.22767053780427, 25.337182455097928, 32.05738791017613, 24.634965229833803, 20.21651837731097, 21.651207180757364, 33.45971338796034, 17.281735993727615, 33.96453060399735, 26.13636481635536, 29.619267552463114, 26.486795967050483, 29.105705780608254, 29.51250950078993, 29.88140504285644, 24.676959168828763, 25.681644221835278, 19.403744358422152, 27.139264577396496, 32.473499516253824, 41.600384136539496, 30.88589423679119, 33.388453336106465, 36.98814102395403, 35.48368470220751, 36.76257675766246, 26.08490076575903, 26.291822630907518, 25.96933759113894, 27.95970198037579, 36.683043174978984, 32.08635304384195, 26.50230917158636, 32.51258387948663, 30.57491740296938, 26.105313819871682, 38.51936967455434, 31.329872206836185, 33.659434244091024, 22.904135146246933, 25.34263540387338, 31.200637394020593, 32.325875925639046, 26.196318114208022, 26.789628654256326, 31.213435313185165, 12.8383547841051, 30.271833342925845, 40

In [90]:
difference = (average_value - 1.324899) * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(current_std * 100000))

 difference from beachmark [pcm]  -2862.5604112004767  with standart deviation [pcm]+- 826.876733431019


In [82]:
Na_estimator = estimators_result[4]

average_value = sum(Na_estimator.collision_sum)/len(Na_estimator.collision_sum)
average_value = average_value / ( zero_point * (7.757166007 -5.754395005) * 0.05)

average_array = np.array(Na_estimator.collision_sum)
average_array = average_array / (zero_point * (7.757166007 -5.754395005) * 0.05)
current_std = get_std(average_array)

In [83]:
difference = (average_value - 0.912273) * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(current_std * 100000))

 difference from beachmark [pcm]  -365.83714791734724  with standart deviation [pcm]+- 278.3244343008635


In [101]:
def save_flux_to_file(estimators, file_name):
    
    
    k = 0
    for estimator in estimators:
        
        result_file_name = file_name + str(k)+".txt"
        
        f = open(result_file_name, "w")
        array_without_zeros = estimator.collision_sum
        for i in range(0, len(array_without_zeros)):
            f.write(" flux "+ str(array_without_zeros[i]) +'\n')
        k += 1
        
        f.close()

In [102]:
file_name = "result_flux"
save_flux_to_file(estimators, file_name)


In [103]:
def save_simulation_results_to_file(k_effective, file_name):
    
    f = open(file_name, "w")
    
    for i in range(0, len(k_effective)):
        f.write(" cycle k effective "+ str(k_effective[i]) + '\n')
            
    
    f.close()


# In[58]:


file_name = "output"
save_simulation_results_to_file(k_effective, file_name)

In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[64]:


#!/usr/bin/env python
# coding: utf-8

# In[1]:


import numpy as np
import matplotlib.pyplot as plt
import math
import random
import h5py
import copy
from scipy.optimize import curve_fit


# In[2]:


class Sig:
    def __init__(self):
        self.number_groups = 0
        self.sig_f = []
        self.capture = []
        self.scattering = []
        self.total = []
        self.number_of_production_neutrons = []
        self.c_value = []
        self.virtual = []
        
    def __init__(self, number_groups, sig_f, sig_c, sig_s, number_of_production_neutrons, sig_t):
        self.number_groups = number_groups
        self.sig_f = sig_f
        self.sig_c = sig_c
        self.sig_s = sig_s      
        self.number_of_production_neutrons = number_of_production_neutrons
        self.sig_t = sig_t
        self.delta_xs = max(sig_t)/2.
        max_total_cs = max(self.sig_t)
        self.virtual = [self.delta_xs]
        
               
    def get_virtual_cs(self):
        max_total_cs = max(self.sig_t)
        self.virtual = [cross_section - max_total_cs + self.delta_xs for cross_section in self.sig_t]
        
    def return_virtual(self):
        return self.virtual
        
    def get_fission_probability(self, energy_group_idx):
        return self.sig_f[energy_group_idx] / self.sig_t[energy_group_idx]
    
    def get_capture_probability(self, energy_group_idx):
        return self.sig_c[energy_group_idx] / self.sig_t[energy_group_idx]
        
    def get_scatter_probability(self, energy_group_idx):
        return self.sig_s[energy_group_idx] / self.sig_t[energy_group_idx]
        


# In[3]:


energy_groups_2 = [0, 2 * 1E10]

class Positon:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
class Direction:
    def __init__(self, ets, phi):
        self.tetta_x = math.sin(ets) * math.cos(phi)
        self.tetta_y = math.sin(ets) * math.sin(phi)
        self.tetta_z = math.cos(ets)  


# In[4]:


class Particle:
    def __init__(self):
        self.coordinates = Positon(0, 0, 0)
        self.direction = Direction(0, 0)
        self.energy = 0
        self.weight = 1
        self.energy_groups = []
        self.terminated = False
        self.path = 0.
        self.last_cell = 0
        
    def set_coordinates(self, x, y, z):
        self.coordinates = Positon(x, y, z)
        
    def set_direction(self, ets, phi):
        self.direction = Direction(ets, phi)
    
    def set_direction_angle(self, angle):
        self.direction = angle
        
    def set_energy_groups(self, energy_groups):
        self.energy_groups = energy_groups
        
    def get_energy_group(self):  
        res = next(x for x, val in enumerate(self.energy_groups)
                                  if val > self.energy)        
        return res - 1
    
    def set_terminated(self):
        self.terminated = True
        
    def is_terminated(self):
        return self.terminated
        
    def get_weight(self):
        return self.weight
    
    def set_weight(self, weight):
        self.weight = weight
    
    def set_particle_deleted(self):
        self.weight = self.weight * 0
        
    def set_particle_fission(self, additional_weight):
        self.weight = self.weight * additional_weight
        
    def set_multiplicity(self, additional_weight):
        self.weight = self.weight * additional_weight
              
    def is_particle_deleted(self):
        return self.weight < 0.0001
    
    def add_path(self, path):
        self.path += path
    
    def get_path(self):
        return self.path
    
    def set_path(self, path):
        self.path = path
        
    def print_direction(self):
        print(str(self.direction.tetta_x) + " " + str(self.direction.tetta_y) + " " + str(self.direction.tetta_z))
        
    def print_coordinates(self):
        print(str(self.coordinates.x) + "  " + str(self.coordinates.y) + " " + str(self.coordinates.z))  
        


# In[5]:


class Plane:
    def __init__(self, A, B, C, D):
        self.A = A
        self.B = B
        self.C = C
        self.D = D
        
    def distance(self, particle):
        vp = (self.A * particle.direction.tetta_x + self.B * particle.direction.tetta_y + 
              self.C * particle.direction.tetta_z)
        
        
        if (abs(vp) < 1e-9):
            return -1
        
        distance = (self.D - self.A * particle.coordinates.x - self.B * particle.coordinates.y 
                    - self.C * particle.coordinates.z) / vp
        
        return distance 
    
    def get_normal(self):
        sq = math.sqrt(self.A * self.A + self.B * self.B + self.C * self.C)
        a_n = self.A / sq
        b_n = self.B / sq
        c_n = self.C / sq
        return [a_n, b_n, c_n]
    

    def get_sign(self, particle):
      
        sign = (self.A * particle.coordinates.x + self.B * particle.coordinates.y +
        self.C * particle.coordinates.z - self.D)
        
        if sign == 0:
            return sign    
            
        if sign < 0:
            return -1
        else:
            return 1
        
    def get_xml(self):
        
        obj_xml = " Plane with A "+ str(self.A) + " B "+ str(self.B) + " C "+ str(self.C) + " D "+ str(self.D)
        
        return obj_xml
    


# In[6]:


class Universe:
    def __init__(self, cells, xs):
        self.cells = cells
        self.xs = xs
        self.delta_xs = 0.
        
    def calculate_delta_xs(self):
        
        delta_xs = 0.
        for xs in self.xs:
            delta_xs = max(delta_xs, max(xs.sig_t))
        self.delta_xs = delta_xs           
            
    def get_delta_xs(self):
        
        return self.delta_xs
    
    def get_xs_by_coordinates(self, particle):
        
        for i in range(0, len(cells)):
            cell = cells[i]
            if cell.is_inside(particle):
                particle.last_cell = i
                return self.xs[i]
        return []
    
    def is_inside(self, particle):
        
        for cell in self.cells:
            if cell.is_inside(particle):
                return True
        return False
        
    
    def is_boudary_reflective(self, particle):
        
        minumum = np.inf
        idx_of_min = 0.
        for i in range(0, len(self.cells)):
            distance, surface_idx = self.cells[i].get_minimum_distance(particle)
            if abs(distance) < minumum:
                minumum = distance
                idx_of_min = i
        if self.cells[idx_of_min].is_boudary_reflective(particle):
            return True
        else:
            return False
        
    def reflect_particle(self,  particle):
        self.cells[particle.last_cell].reflect_particle(particle)       


# In[7]:


def set_random_direction(particle):
    r = 1.0
    r1 = 1.0
    while (r**2 + r1**2 > 1.0):
        r = 2. * random.uniform(0, 1) - 1.
        r1 = 2. * random.uniform(0, 1) - 1.
        
    direction_x = 2.0 * r**2 + 2. * r1**2 - 1.0  
    particle.direction.tetta_x = direction_x
    particle.direction.tetta_y = r * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))
    particle.direction.tetta_z = r1 * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))


# In[8]:


import random

def make_initial_sources(number_of_paricles, box_size, zero_point, energy=10.0e6):
    
    step_x = box_size[0]/number_of_paricles
    step_y = box_size[1]/number_of_paricles
    step_z = box_size[2]/number_of_paricles
    
    x_coord = zero_point[0]
    y_coord = zero_point[1]
    z_coord = zero_point[2]
    
    sources = []
    for i in range(0, number_of_paricles):
        for j in range(0, number_of_paricles):
            for k in range(0, number_of_paricles):
                current_particle = Particle()
                current_particle.set_coordinates(x_coord + k * step_x, y_coord + j * step_y, z_coord+ i * step_z)
                set_random_direction(current_particle)
                current_particle.energy = energy
                current_particle.set_energy_groups(energy_groups_2)
                sources.append(current_particle)

    return sources


# In[9]:


def get_free_path(particle, delta_xs):
    
    energy_group_idx = particle.get_energy_group()   
    sig_t = delta_xs
    free_path = -math.log(random.uniform(0, 1)) / sig_t
    
    return free_path  


# In[10]:


class Cell:
    def __init__(self):
        self.surfaces = []
        self.boundaries_type = []
        self.signs = []
        self.size = 0
        
    def set_boundaries_type(self, boundaries_type):
        self.boundaries_type = boundaries_type        
        
    def set_box_sizes(self, xm_size, ym_size, zm_size, x_size, y_size, z_size):
        self.xm_size = xm_size
        self.ym_size = ym_size
        self.zm_size = zm_size
        
        self.x_size = x_size
        self.y_size = y_size
        self.z_size = z_size
        
    def set_zero_point(self, x_0, y_0, z_0):
        self.x_0 = x_0
        self.y_0 = y_0
        self.z_0 = z_0
        
    def __init__(self, surfaces, signs):
        self.surfaces = surfaces
        self.signs = signs
        self.size = len(self.signs)
           
    def is_inside(self, particle):
        
        is_inside = True
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                is_inside = False
                
        return is_inside
    
    def is_boudary_reflective(self, particle):
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                if self.boundaries_type[i] == 'reflective':
                    return True
                else:
                    return False
        
    
    def get_minimum_distance(self, particle):
        
        minimum_distance = math.inf
        surface_idx = 0
        for i in range(0, len(self.surfaces)):
            current_distance = abs(self.surfaces[i].distance(particle))
            if current_distance > 0:
                minimum_distance = min(minimum_distance, current_distance)
                surface_idx = i
        return minimum_distance, surface_idx
    
    def get_xml(self):
        
        xml_obj = []
        for i in range(0, self.size):
            xml_obj.append(surfaces[i].get_xml())
            
        return xml_obj
    
    def reflect_particle(self, particle):
        
        if particle.coordinates.x > (self.x_size + self.x_0):
            delta = particle.coordinates.x % self.x_size
            particle.coordinates.x = self.xm_size + math.fabs(delta)
        
        if particle.coordinates.y > (self.y_size + self.y_0):
            
            delta = particle.coordinates.y % self.y_size
            particle.coordinates.y = self.ym_size + math.fabs(delta)
            
            
        if particle.coordinates.z > (self.z_size + self.z_0):
            
            delta = particle.coordinates.z % self.z_size
            particle.coordinates.z = self.zm_size + math.fabs(delta)
        
        
        if particle.coordinates.y < (self.ym_size + self.y_0):
            delta = particle.coordinates.y % self.ym_size
            particle.coordinates.y = self.y_size - math.fabs(delta)
            
            
        if particle.coordinates.x < (self.xm_size + self.x_0):
            delta = particle.coordinates.x % self.xm_size
            particle.coordinates.x = self.x_size - math.fabs(delta)
            
        if particle.coordinates.z < (self.zm_size + self.z_0):
            delta = particle.coordinates.z % self.zm_size
            particle.coordinates.z = self.z_size - math.fabs(delta)
                    


# In[11]:


def move_particle(particle, t):
    new_x = particle.direction.tetta_x * t + particle.coordinates.x
    new_y = particle.direction.tetta_y * t + particle.coordinates.y
    new_z = particle.direction.tetta_z * t + particle.coordinates.z
    particle.set_coordinates(new_x, new_y, new_z)


# In[13]:


def is_collision_virual(particle, c_s, delta_xs):
    
    energy_group_idx = particle.get_energy_group()
    random_number = random.uniform(0, 1)
    
    virtual_cs = delta_xs
    
    total_cs = c_s.sig_t[energy_group_idx]
    
    if total_cs/virtual_cs < random_number:
        return True
    else:
        return False


# In[14]:


def process_virtual_collision(particle, free_path):
    particle.add_path(free_path)


# In[15]:


def process_real_collision(particle, free_path, c_s):
    
    energy_group_idx = particle.get_energy_group()
    weight_before_collision = particle.get_weight()
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]   
    capture_probability = c_s.get_capture_probability(energy_group_idx)
    scatter_probability = c_s.get_scatter_probability(energy_group_idx)  
    fission_probability = c_s.get_fission_probability(energy_group_idx)  
    
    
    type_collision = np.random.choice(['capture', 'scatter', 'fission'], p=[capture_probability, scatter_probability,
                                                                            fission_probability])
    if type_collision == 'capture':
        particle.set_terminated()
        particle.set_weight(0.)
        
    if type_collision == 'scatter':      
        set_random_direction(particle)
        
    if type_collision == 'fission':    
        particle.set_particle_fission(number_of_production_neutrons)
        particle.set_terminated()
        
    return weight_before_collision


# In[16]:


def delete_absorpbed_paricles(particles):
    
    existing_particles = []
    for i in range(0, len(particles)):
        particle = particles[i]
        if particle.get_weight() > 0.000001:
            existing_particles.append(particle)
                     
    return existing_particles


# In[17]:


def process_one_particle_history(particle, universe, estimators):
    
    sum_collisions = 0.
    universe.calculate_delta_xs()
    delta_xs = universe.get_delta_xs()
    
    while not particle.is_terminated():
        current_xs = universe.get_xs_by_coordinates(particle)
        free_path = get_free_path(particle, delta_xs)        
        move_particle(particle, free_path)
        current_xs = universe.get_xs_by_coordinates(particle)
        
        if not universe.is_inside(particle):
            if universe.is_boudary_reflective(particle):
                universe.reflect_particle(particle)
                current_xs = universe.get_xs_by_coordinates(particle)

            else:
                particle.set_terminated()
                particle.set_weight(0.)
                return particle, sum_collisions
                
        for k in estimators:
            k.add_collision(particle)
        
        if is_collision_virual(particle, current_xs, delta_xs):
            process_virtual_collision(particle, free_path)
        else:
            process_real_collision(particle, free_path, current_xs)
        
            
    return particle, sum_collisions


# In[18]:


def get_weights(particles):
    
    weights = []
    for i in range(0, len(particles)):
        weights.append(particles[i].weight)
        
    return weights


# In[19]:


def make_sources(particles):
    for i in range(0, len(particles)):
        particles[i].terminated = False
        set_random_direction(particles[i])       


# In[20]:


def splitting_secound_version(particles):
    
    initial_size = len(particles)
    
    for i in range(0, initial_size):
        particle = particles[i]
        current_weight = particle.get_weight()
        if current_weight > 1:
            n_value = math.floor(current_weight)
            random_value = random.uniform(0, 1)
            if current_weight - n_value >= random_value:
                n_value += 1
            if n_value > 1:
                particle.set_weight(current_weight / n_value)
                for j in range(0, n_value - 1):
                    particles.append(copy.deepcopy(particle))


# In[21]:


def normalise_weights(particles, batch_size):

    sum_weights = 0.
    
    for i in range(0, len(particles)):
        sum_weights += particles[i].get_weight()

    for i in range(0, len(particles)):
        particles[i].weight = (particles[i].weight * batch_size) / sum_weights   
        
    return particles


class Flux_estimator:
    def __init__(self, universe):
        self.boundaries = universe
        self.collision_sum = [0]
        
    def add_collision(self, particle):
        if self.boundaries.is_inside(particle):
            self.collision_sum[-1]  += particle.get_weight()
            



def russian_roulette(weights_previous, particles_current):
    
    for i in range(0, len(particles_current)):
        particle = particles_current[i]
        if particle.weight > 0 and particle.weight < 0.5 and particle.weight < weights_previous[i]:
            probability_terminate = 1. - particle.weight / weights_previous[i]
            random_number = random.uniform(0, 1)
            if probability_terminate >= random_number:
                particle.set_weight(0.)
            else:
                particle.set_weight(weights_previous[i])


# In[23]:


def terminate_outside_particles(batch_particles, universe):
    
    for j in range(0, len(batch_particles)):
        particle = batch_particles[j]
        if not universe.is_inside(particle):
            print("ERROR! outside particle")


# In[24]:


def reset_estimators(estimators, weight_previos, volume, sum_collisions, c_s):
    
    sig_t = c_s.sig_t[0]

    for estimator in estimators:
        estimator.collision_sum[-1] = estimator.collision_sum[-1]
        estimator.collision_sum.append(0.)  


# In[25]:


def get_std(values):
    
    current_std = np.std(values)/np.sqrt((len(values) - 1.))
  
    return current_std   


# In[26]:


def calculate_k_effective(idx, weights, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, std_k_effective):
    
    keff_cycle = sum(weights) / initial_size
    if idx > number_inactive:
        k_effective.append(keff_cycle)
        
    if idx > number_inactive + 1:
    
        k_effective_exp.append(sum(k_effective) / len(k_effective))
        
        std_k_effective_current = get_std(k_effective)
            
        std_k_effective.append(std_k_effective_current)

        print(" keff_cycle , k_effective_exp, std_k_effective " + str(keff_cycle) + "   " + str(k_effective_exp[-1]) +
                                    "  "+ str(std_k_effective[-1]))


# In[27]:


import statistics
import time


def simulation_black_boundaries(universe, number_interations, number_inactive, number_of_particles, c_s, estimators,
                                volume=1):
    
    random.seed(time.time())
    k_effective = []
    k_effective_std = []
    k_effective_exp = []
    flux = []
    flux_exp = []
    num1 = random.randint(0,9)
    
    print(" num1  " + str(num1))
    
    box_size = [1., 1., 1.]
    zero_point = [0.1, 0.1, 0.1]
    initial_sources = make_initial_sources(number_of_particles, box_size, zero_point, energy=10.0e6)
    initial_size = len(initial_sources)
    
    weights_previous = [1.] * len(initial_sources)
    
    for i in range(0, number_interations):
        print("i == " + str(i))
        
        make_sources(initial_sources)
        batch_size = len(initial_sources)
        
        batch_particles = []

        for j in range(0, batch_size):
   
            particle = initial_sources[j]
            terminate_particle, sum_collisions = process_one_particle_history(particle, universe, estimators)
            batch_particles.append(terminate_particle)
        
        russian_roulette(weights_previous, batch_particles)
        
        batch_particles = delete_absorpbed_paricles(batch_particles)
        
        splitting_secound_version(batch_particles)
        
        weights_cycle = get_weights(batch_particles)
        
        calculate_k_effective(i, weights_cycle, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, k_effective_std)
        
        
        reset_estimators(estimators, weights_previous, volume, sum_collisions, c_s) 
      

        batch_particles = normalise_weights(batch_particles, initial_size)  

        initial_sources = batch_particles
        weights_previous = get_weights(batch_particles)

            
    return k_effective, k_effective_exp, k_effective_std, estimators


# In[28]:


cs_fission_u235 = [0.06922744]

cs_capture_u235 = [0.01013756]

cs_scattering_u235 = [0.328042]

cs_total_u235 = [0.407407]

cs_production_neutrons_u235 = [2.50]


u235 = Sig(1, cs_fission_u235, cs_capture_u235, cs_scattering_u235, 
                       cs_production_neutrons_u235,  cs_total_u235)


cs_fission_fe = [0.]

cs_capture_fe = [0.00046512]

cs_scattering_fe = [0.23209488]

cs_total_fe = [0.23256]

cs_production_neutrons_fe = [0.]

fe = Sig(1, cs_fission_fe, cs_capture_fe, cs_scattering_fe, 
                       cs_production_neutrons_fe,  cs_total_fe)


cs_fission_na = [0.]

cs_capture_na = [0.0]

cs_scattering_na = [0.086368032]

cs_total_na = [0.086368032]

cs_production_neutrons_na = [0.]

na = Sig(1, cs_fission_na, cs_capture_na, cs_scattering_na, 
                       cs_production_neutrons_na,  cs_total_na)



energy_groups_2 = [0, 2 * 1E10]


# In[29]:


x_5 = Plane(1, 0, 0, 1000)
x_m5 = Plane(1, 0, 0, -1000)

y_5 = Plane(0, 1, 0, 0.317337461)

y_m5 = Plane(0, 1, 0, 0.)


z_5 = Plane(0, 0, 1, 1000)
z_m5 = Plane(0, 0, 1, -1000)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

signs = [-1, +1, -1, +1, -1, +1]
slab_fe = Cell(surfaces, signs)

boundaries_type = ["reflective", "reflective","black", "black", "reflective", "reflective"]


slab_fe.set_boundaries_type(boundaries_type)
slab_fe.set_box_sizes(-1000, 10900., -1000, 1000, 20000., 1000 )
slab_fe.set_zero_point(0., 0., 0.)


# In[30]:


x_5 = Plane(1, 0, 0, 1000)
x_m5 = Plane(1, 0, 0, -1000)

y_5 = Plane(0, 1, 0, 5.437057544)

y_m5 = Plane(0, 1, 0, 0.317337461)


z_5 = Plane(0, 0, 1, 1000)
z_m5 = Plane(0, 0, 1, -1000)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

signs = [-1, +1, -1, +1, -1, +1]
slab_u235 = Cell(surfaces, signs)

boundaries_type = ["reflective", "reflective","black", "black", "reflective", "reflective"]


slab_u235.set_boundaries_type(boundaries_type)
slab_u235.set_box_sizes(-1000, -10000., -1000, 1000, 10000., 1000 )
slab_u235.set_zero_point(0., 0., 0.)


# In[31]:


x_5 = Plane(1, 0, 0, 1000)
x_m5 = Plane(1, 0, 0, -1000)

y_5 = Plane(0, 1, 0, 5.754395005)

y_m5 = Plane(0, 1, 0, 5.437057544)


z_5 = Plane(0, 0, 1, 1000)
z_m5 = Plane(0, 0, 1, -1000)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

signs = [-1, +1, -1, +1, -1, +1]
slab_fe2 = Cell(surfaces, signs)

boundaries_type = ["reflective", "reflective","black", "black", "reflective", "reflective"]


slab_fe2.set_boundaries_type(boundaries_type)
slab_fe2.set_box_sizes(-1000, -10000., -1000, 1000, 10000., 1000 )
slab_fe2.set_zero_point(0., 0., 0.)


# In[32]:


x_5 = Plane(1, 0, 0, 1000)
x_m5 = Plane(1, 0, 0, -1000)

y_5 = Plane(0, 1, 0, 7.757166007)

y_m5 = Plane(0, 1, 0, 5.754395005)


z_5 = Plane(0, 0, 1, 1000)
z_m5 = Plane(0, 0, 1, -1000)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

signs = [-1, +1, -1, +1, -1, +1]
slab_na = Cell(surfaces, signs)

boundaries_type = ["reflective", "reflective","black", "black", "reflective", "reflective"]


slab_na.set_boundaries_type(boundaries_type)
slab_na.set_box_sizes(-1000, -10000.,  -1000, 1000, 10000., 1000 )
slab_na.set_zero_point(0., 0., 0.)


# In[33]:


cells = [slab_fe, slab_u235, slab_fe2, slab_na]


# In[34]:


estimators = []


# In[35]:


materials = [fe, u235, fe, na]


# In[36]:


U235_Reactor = Universe(cells, materials)


# In[37]:


# In[65]:


estimators = []

# zero

x_5 = Plane(1, 0, 0, 10000)
x_m5 = Plane(1, 0, 0, -10000)

delta = 0.317337461 * 0.1


y_5 = Plane(0, 1, 0,  delta)

y_m5 = Plane(0, 1, 0, 0)


z_5 = Plane(0, 0, 1, 10000)
z_m5 = Plane(0, 0, 1, -10000)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

signs = [-1, +1, -1, +1, -1, +1]
current_slub = Cell(surfaces, signs)

estimators.append(Flux_estimator(current_slub))


# In[66]:


# Fe-U

x_5 = Plane(1, 0, 0, 10000)
x_m5 = Plane(1, 0, 0, -10000)

delta = 0.317337461 * 0.05


y_5 = Plane(0, 1, 0,  0.317337461 + delta)

y_m5 = Plane(0, 1, 0, 0.317337461 - delta)


z_5 = Plane(0, 0, 1, 10000)
z_m5 = Plane(0, 0, 1, -10000)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

signs = [-1, +1, -1, +1, -1, +1]
current_slub = Cell(surfaces, signs)

estimators.append(Flux_estimator(current_slub))


# In[67]:


# U-Fe

x_5 = Plane(1, 0, 0, 10000)
x_m5 = Plane(1, 0, 0, -10000)

delta = (0.317337461) * 0.01


y_5 = Plane(0, 1, 0,  5.437057544 + delta)

y_m5 = Plane(0, 1, 0, 5.437057544 - delta)


z_5 = Plane(0, 0, 1, 10000)
z_m5 = Plane(0, 0, 1, -10000)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

signs = [-1, +1, -1, +1, -1, +1]
current_slub = Cell(surfaces, signs)

estimators.append(Flux_estimator(current_slub))


# In[68]:


# Fe-Na

x_5 = Plane(1, 0, 0, 10000)
x_m5 = Plane(1, 0, 0, -10000)

delta = (5.754395005 - 5.437057544)  * 0.02


y_5 = Plane(0, 1, 0,  5.754395005 + delta)

y_m5 = Plane(0, 1, 0, 5.754395005 - delta)


z_5 = Plane(0, 0, 1, 10000)
z_m5 = Plane(0, 0, 1, -10000)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

signs = [-1, +1, -1, +1, -1, +1]
current_slub = Cell(surfaces, signs)

estimators.append(Flux_estimator(current_slub))


# In[69]:


# Na

x_5 = Plane(1, 0, 0, 10000)
x_m5 = Plane(1, 0, 0, -10000)

delta = (7.757166007 -5.754395005) * 0.05


y_5 = Plane(0, 1, 0,  7.757166007)

y_m5 = Plane(0, 1, 0, 7.757166007 - delta)


z_5 = Plane(0, 0, 1, 10000)
z_m5 = Plane(0, 0, 1, -10000)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

signs = [-1, +1, -1, +1, -1, +1]
current_slub = Cell(surfaces, signs)

estimators.append(Flux_estimator(current_slub))


# In[70]:



test_number_of_particles = 10
test_number_interations = 100
test_number_inactive = 10


# In[38]:


k_effective, k_effective_exp, k_effective_std, estimators_result = simulation_black_boundaries(U235_Reactor,  
                           test_number_interations,  test_number_inactive, 
                          test_number_of_particles, u235, estimators)


# In[40]:


def save_simulation_results_to_file(k_effective, file_name):
    
    f = open(file_name, "w")
    
    for i in range(0, len(k_effective)):
        f.write(" cycle k effective "+ str(k_effective[i]) + '\n')
            
    
    f.close()


# In[42]:


file_name = "output.txt"
save_simulation_results_to_file(k_effective, file_name)


# In[71]:


difference = (k_effective_exp[-1] - 1.) * 100000
k_effective_std = get_std(k_effective)
standart_deviation = k_effective_std * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(standart_deviation))


# In[72]:


zero_estimator = estimators_result[0]

average_value = sum(zero_estimator.collision_sum)/len(zero_estimator.collision_sum)
zero_point = average_value / (0.317337461 * 0.05 *2)
print(zero_point)


# In[92]:


print(average_value)


# In[73]:


Fe_U_estimator = estimators_result[1]

average_value = sum(Fe_U_estimator.collision_sum)/len(Fe_U_estimator.collision_sum)
average_value = average_value / ( zero_point * 0.317337461 * 0.05 * 2.)

average_array = np.array(Fe_U_estimator.collision_sum)
average_array = average_array / (zero_point * 0.317337461 * 0.05 * 2.)
current_std = get_std(average_array)


# In[91]:


U_Fe_estimator = estimators_result[2]

start = 10
end = len(U_Fe_estimator.collision_sum) - 1

average_value = sum(U_Fe_estimator.collision_sum[start: end])/len(U_Fe_estimator.collision_sum[start: end])
average_value = average_value / (zero_point * 2. * 0.317337461 * 0.01)


average_array = np.array(U_Fe_estimator.collision_sum[start: end])
average_array = average_array / (zero_point * 2. * 0.317337461 * 0.01)
current_std = get_std(average_array)


# In[93]:


difference = (average_value - 1.49712) * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(current_std * 100000))


# In[87]:


U_Fe_estimator = estimators_result[3]
start = 10
end = len(U_Fe_estimator.collision_sum) - 1
average_value = sum(U_Fe_estimator.collision_sum[start: end])/len(U_Fe_estimator.collision_sum[start: end])
average_value = average_value / (zero_point * (5.754395005 - 5.437057544)  * 0.04)


average_array = np.array(U_Fe_estimator.collision_sum[start: end])
average_array = average_array / (zero_point * (5.754395005 - 5.437057544)  * 0.04)
current_std = get_std(average_array)


# In[89]:


print(U_Fe_estimator.collision_sum[start: end])


# In[90]:


difference = (average_value - 1.324899) * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(current_std * 100000))


# In[82]:


Na_estimator = estimators_result[4]

average_value = sum(Na_estimator.collision_sum)/len(Na_estimator.collision_sum)
average_value = average_value / ( zero_point * (7.757166007 -5.754395005) * 0.05)

average_array = np.array(Na_estimator.collision_sum)
average_array = average_array / (zero_point * (7.757166007 -5.754395005) * 0.05)
current_std = get_std(average_array)


# In[83]:


difference = (average_value - 0.912273) * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(current_std * 100000))


# In[101]:


def save_flux_to_file(estimators, file_name):
    
    
    k = 0
    for estimator in estimators:
        
        result_file_name = file_name + str(k)+".txt"
        
        f = open(result_file_name, "w")
        array_without_zeros = estimator.collision_sum
        for i in range(0, len(array_without_zeros)):
            f.write(" flux "+ str(array_without_zeros[i]) +'\n')
        k += 1
        
        f.close()


# In[102]:


file_name = "result_flux"
save_flux_to_file(estimators, file_name)


# In[103]:


def save_simulation_results_to_file(k_effective, file_name):
    
    f = open(file_name, "w")
    
    for i in range(0, len(k_effective)):
        f.write(" cycle k effective "+ str(k_effective[i]) + '\n')
            
    
    f.close()


# In[58]:


file_name = "output"
save_simulation_results_to_file(k_effective, file_name)


# In[ ]:





 num1  9
i == 0
i == 1
i == 2
i == 3
i == 4
i == 5
i == 6
i == 7
i == 8
i == 9
i == 10
i == 11
i == 12
 keff_cycle , k_effective_exp, std_k_effective 1.0282755422773875   1.046215898414064  0.01794035613667644
i == 13
 keff_cycle , k_effective_exp, std_k_effective 1.033416819863008   1.0419495388970452  0.01120211064779238
i == 14
 keff_cycle , k_effective_exp, std_k_effective 1.034365347919248   1.040053491152596  0.008144853499721345
i == 15
 keff_cycle , k_effective_exp, std_k_effective 0.939088837828128   1.0198605604877025  0.021155557944813316
i == 16
 keff_cycle , k_effective_exp, std_k_effective 1.0050865758675134   1.017398229717671  0.01744806085095456
i == 17
 keff_cycle , k_effective_exp, std_k_effective 0.9793554204056761   1.0119635426731002  0.015715892321378003
i == 18
 keff_cycle , k_effective_exp, std_k_effective 0.9300161593680458   1.0017201197599683  0.017034367219539874
i == 19
 keff_cycle , k_effective_exp, std_k_effective 1.0134760341941864   1.0030263324748814 

 keff_cycle , k_effective_exp, std_k_effective 1.033851152242476   0.9973916030455605  0.004557084350840447
i == 83
 keff_cycle , k_effective_exp, std_k_effective 1.0253237332736906   0.9977742349664939  0.004510483962541119
i == 84
 keff_cycle , k_effective_exp, std_k_effective 1.0493987584854074   0.9984718636626955  0.004503476540754079
i == 85
 keff_cycle , k_effective_exp, std_k_effective 0.9709192815033725   0.998104495900571  0.004458186327834142
i == 86
 keff_cycle , k_effective_exp, std_k_effective 0.9700769474377875   0.997735712368166  0.004414565563980126
i == 87
 keff_cycle , k_effective_exp, std_k_effective 1.0121042994146554   0.9979223173947439  0.004360850677271897
i == 88
 keff_cycle , k_effective_exp, std_k_effective 0.9962661196197163   0.9979010840899358  0.004304631647470189
i == 89
 keff_cycle , k_effective_exp, std_k_effective 0.9148731436033106   0.9968500975014977  0.004377820943259449
i == 90
 keff_cycle , k_effective_exp, std_k_effective 0.9730789277491818  